In [ ]:
import numpy
import pandas as pd
import json
import requests


In [ ]:
# Load the JSON data from the file
with open('sarcasm_data.json', 'r') as file:
    data = dict(json.load(file))


# df = pd.DataFrame(list(data.items()), columns=['Key', 'Values'])
df = pd.DataFrame(data).T
df=df.reset_index()
df=df.rename(columns={'index': 'video'})
df.loc[:10,['video','context','utterance','sarcasm']]


,video,context,utterance,sarcasm
0,1_60,[I never would have identified the fingerprint...,It's just a privilege to watch your mind at work.,True
1,1_70,[This is one of my favorite places to kick bac...,I don't think I'll be able to stop thinking ab...,True
2,1_80,"[Here we go. Pad thai, no peanuts., But does i...","Since it's not bee season, you can have my epi...",False
3,1_90,[A marathon? How many Superman movies are ther...,"Lois Lane is falling, accelerating at an initi...",False
4,1_105,"[Great Caesar's ghost, look at this place., So...",I'm just inferring this is a couch because the...,True
5,1_162,"[On the other hand, no arousal., None?, None.,...","Sheldon, if you were a robot, and I knew and y...",False
6,1_175,"[Oh, good Lord., God, that's a good song.]",If you're compiling a mix CD for a double suic...,True
7,1_182,"[How do I look?, Could you be more specific?, ...",No. The dark crescent-shaped patterns under yo...,True
8,1_213,"[You know, I am not going to enjoy this party....","How's this? ""Pleased to meet you, Dr. Gablehau...",True
9,1_276,[He switched over to high-energy radiation res...,"Yeah, my parents felt that naming me Leonard a...",True


In [ ]:
def gpt4(prompt):
  API_KEY = ""
  API_ENDPOINT = "https://api.openai.com/v1/chat/completions"

  headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {API_KEY}",
  }


  messages = [
    {"role": "user", "content": prompt}
  ]
  data = {
      "model": 'gpt-4',
      "messages": messages,
      "temperature": 0,
      "max_tokens":50
  }

  response = requests.post(API_ENDPOINT, headers=headers, data=json.dumps(data))

  if response.status_code == 200:
      return response.json()["choices"][0]["message"]["content"]
  else:
      raise Exception(f"Error {response.status_code}: {response.text}")

In [ ]:
def gpt_predict(context,utterance):

  # Example usage
  prompt_template='''You are given a context and an utterance. Your job is detect whether there is sarcasm in the utterance.

  Below are a few examples
  Context: I never would have identified the fingerprints of string theory in the aftermath of the Big Bang.,My apologies. What's your plan?
  Utterance: It's just a privilege to watch your mind at work.
  Sarcasm: true

  Context: Here we go. Pad thai, no peanuts.,But does it have peanut oil?,I'm not sure. Everyone keep an eye on Howard in case he starts to swell up.
  Utterance: Since it's not bee season, you can have my epinephrine.
  Sarcasm: false

  Context: Oh, good Lord.,God, that's a good song.
  Utterance: If you're compiling a mix CD for a double suicide. Oh, I hope that scratching post is for you.
  Sarcasm: true

  Context: Here we go. Pad thai, no peanuts.,But does it have peanut oil?,I'm not sure. Everyone keep an eye on Howard in case he starts to swell up.
  Utterance: Since it's not bee season, you can have my epinephrine.
  Sarcasm: false

  Context: This is Dr. Sheldon Cooper.,Yeah, I need to cancel my membership to the planetarium.
  Utterance: Well, I'm sorry, too, but there's just no room for you in my wallet.
  Sarcasm: true

  Context: {context}
  Utterance: {utterance}
  Sarcasm: '''

  response = gpt4(prompt_template.format(context=context,utterance=utterance))
  return response



In [ ]:
video_list=['1_213', '1_427', '1_507', '1_1001', '1_3293', '1_4031', '1_5156', '1_5496', '2_12', '2_53', '2_102', '2_128', '2_280', '2_410', '2_370', '2_376', '2_429', '2_450', '2_500', '2_523', '2_550', '2_560', '2_607', '2_612', '2_626', '1_182', '1_672', '1_1296', '1_2669', '1_3573']
# video_list=['1_60']
sample=df[df['video'].isin(video_list)]
sample=sample.copy(deep=True)

In [ ]:
#with context
sample['GPT_with_context']=sample.apply(lambda x: gpt_predict(context=x['context'],utterance=x['utterance']), axis=1)

In [ ]:
#without context
sample['GPT_without_context']=sample.apply(lambda x: gpt_predict(context="",utterance=x['utterance']), axis=1)

In [ ]:
sample['sarcasm']=sample['sarcasm'].apply(str).apply(str.lower)
print(sample[['video','GPT_with_context','GPT_without_context','sarcasm']])
# print(sample)

      video GPT_with_context GPT_without_context sarcasm
7     1_182             true                true    true
8     1_213             true                true    true
12    1_427             true               false    true
16    1_507             true                true    true
20    1_672            false               false   false
22   1_1001             true                true   false
29   1_1296             true                true    true
62   1_2669            false               false   false
79   1_3293             true                true    true
86   1_3573             true                true    true
96   1_4031             true                true    true
119  1_5156             true                true    true
124  1_5496             true                true    true
281   2_370             true               false    true
335   2_450             true                true    true
351   2_280             true                true    true
359   2_376            false   

In [ ]:
correct_with_context = (sample['sarcasm'] == sample['GPT_with_context'])
correct_without_context = (sample['sarcasm'] == sample['GPT_without_context'])
# print(correct)
print("Total correct: ",str(correct_with_context.sum()))
print("Total correct without context: ",str(correct_without_context.sum()))

Total correct:  24
Total correct without context:  22
